In [13]:
import re
import random
import os
import csv

# --- EXCEPTION CLASSES ---
class InvalidInputError(Exception):
    pass

class TaskNotFoundError(Exception):
    pass

class EmployeeNotFoundError(Exception):
    pass

class ClientNotFoundError(Exception):
    pass

class Address:
    def __init__(self, mail_id, phone_number, house_no=None, building_number=None,
                 road_number=None, street_name=None, land_mark=None, city=None,
                 state=None, zip_code=None):
        self.mail_id = self._validate_mail_id(mail_id)
        self.phone_number = self._validate_phone_number(phone_number)
        self.house_no = house_no
        self.building_number = building_number
        self.road_number = road_number
        self.street_name = street_name
        self.land_mark = land_mark
        self.city = self._validate_city_state(city, 'city')
        self.state = self._validate_city_state(state, 'state')
        self.zip_code = self._validate_zip_code(zip_code)
    
    def _validate_mail_id(self, email):
        if not re.fullmatch(r"^[a-zA-Z0-9._-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,4}$", email):
            raise InvalidInputError("Invalid email format.")
        return email

    def _validate_phone_number(self, phone):
        if not re.fullmatch(r"^(?:\+91|)\d{10}$", phone):
            raise InvalidInputError("Phone number must be 10 digits or 10 digits with '+91'.")
        return phone
    
    def _validate_city_state(self, name, field_name):
        if name and not re.fullmatch(r"^[a-zA-Z\s]+$", name):
            raise InvalidInputError(f"The {field_name} can only contain alphabets.")
        return name
        
    def _validate_zip_code(self, code):
        if code and not re.fullmatch(r"^\d+$", str(code)):
            raise InvalidInputError("Zip code must contain only digits.")
        return code

    def __repr__(self):
        return (f"Address(mail_id='{self.mail_id}', phone_number='{self.phone_number}', "
                f"city='{self.city}', state='{self.state}', zip_code='{self.zip_code}')")

# --- TASK MANAGEMENT CLASSES ---
class Task:

    def __init__(self, task_id, task_name, chargeable, rate_card=None):
        self.task_id = task_id
        self.task_name = self._validate_task_name(task_name)
        self.chargeable = self._validate_chargeable(chargeable)
        self.rate_card = self._validate_rate_card(rate_card, self.chargeable)

    def _validate_task_name(self, name):
        if not re.fullmatch(r"^[a-zA-Z0-9\s\-_]+$", name):
            raise InvalidInputError("Task name can only contain alphabets, digits, '-', '_', and spaces.")
        return name

    def _validate_chargeable(self, chargeable):
        if isinstance(chargeable, str):
            if chargeable.lower() in ['true', 'yes']:
                return True
            elif chargeable.lower() in ['false', 'no']:
                return False
            else:
                raise InvalidInputError("Chargeable value must be 'True', 'False', 'Yes', or 'No'.")
        elif isinstance(chargeable, bool):
            return chargeable
        else:
            raise InvalidInputError("Chargeable value must be a boolean or a string representation.")

    def _validate_rate_card(self, rate_card, is_chargeable):
        if is_chargeable:
            if not isinstance(rate_card, (int, float)):
                try:
                    rate_card = float(rate_card)
                except (ValueError, TypeError):
                    raise InvalidInputError("Rate card must be an integer or a float value for chargeable tasks.")
            if rate_card < 0:
                raise InvalidInputError("Rate card cannot be a negative value.")
        else:
            if rate_card is not None and rate_card != 0:
                print("Warning: Rate card is ignored for non-chargeable tasks.")
            return None
        return rate_card

    def to_dict(self):
        return {
            "task_id": self.task_id,
            "task_name": self.task_name,
            "chargeable": self.chargeable,
            "rate_card": self.rate_card
        }

    def __repr__(self):
        return (f"Task(task_id='{self.task_id}', task_name='{self.task_name}', "
                f"chargeable={self.chargeable}, rate_card={self.rate_card})")

class TaskManager:

    def __init__(self, storage_file="/Users/kaarunyalakshmanchinthalapudi/Downloads/tasks.csv"):
        self.tasks = {}
        self.storage_file = storage_file
        print(f"Loading tasks from {self.storage_file}...")

    def _save_tasks_to_file(self):
        print(f"Saving tasks to {self.storage_file}...")

    def _generate_task_id(self):
        while True:
            task_id = f"TSK_{random.randint(100000, 999999)}"
            if task_id not in self.tasks:
                return task_id

    def create_task_from_csv(self, file_path="/Users/kaarunyalakshmanchinthalapudi/Downloads/tasks.csv"):
        if not os.path.exists(file_path):
            print(f"Error: CSV file not found at {file_path}")
            return
        
        try:
            with open(file_path, 'r', encoding='utf-8') as csvfile:
                reader = csv.reader(csvfile)
                for i, row in enumerate(reader):
                    if len(row) < 3:
                        print(f"Skipping invalid row {i+1} from CSV: {row}")
                        continue
                    try:
                        task_id = self._generate_task_id()
                        task_name, chargeable, rate_card = row[0], row[1], row[2]
                        new_task = Task(task_id, task_name, chargeable, rate_card)
                        self.tasks[new_task.task_id] = new_task
                        print(f"Task '{new_task.task_name}' created with ID {new_task.task_id} from CSV.")
                    except (InvalidInputError, IndexError) as e:
                        print(f"Error creating task from CSV row {i+1}: {e}")
            self._save_tasks_to_file()
        except Exception as e:
            print(f"An unexpected error occurred while reading the CSV file: {e}")

    def create_task(self, task_name, chargeable, rate_card=None):
        try:
            task_id = self._generate_task_id()
            new_task = Task(task_id, task_name, chargeable, rate_card)
            self.tasks[new_task.task_id] = new_task
            self._save_tasks_to_file()
            return new_task
        except InvalidInputError as e:
            print(f"Error creating task: {e}")
            return None

    def update_task(self, task_id, task_name=None, chargeable=None, rate_card=None):
        if task_id not in self.tasks:
            raise TaskNotFoundError(f"Task with ID {task_id} not found.")
        task = self.tasks[task_id]
        
        try:
            if task_name is not None:
                task.task_name = task._validate_task_name(task_name)
            if chargeable is not None:
                new_chargeable = task._validate_chargeable(chargeable)
                task.chargeable = new_chargeable
                if rate_card is not None:
                    task.rate_card = task._validate_rate_card(rate_card, new_chargeable)
                elif not new_chargeable:
                    task.rate_card = None
            elif rate_card is not None and task.chargeable:
                task.rate_card = task._validate_rate_card(rate_card, task.chargeable)
            
            self._save_tasks_to_file()
            print(f"Task {task_id} updated successfully.")
            return task
        except InvalidInputError as e:
            print(f"Error updating task {task_id}: {e}")
            return None

    def delete_task(self, task_id):
        if task_id not in self.tasks:
            raise TaskNotFoundError(f"Task with ID {task_id} not found.")
        del self.tasks[task_id]
        self._save_tasks_to_file()
        print(f"Task with ID {task_id} deleted successfully.")

    def search_task(self, task_id):
        return self.tasks.get(task_id)

    def list_all_tasks(self):
        if not self.tasks:
            print("No tasks are available in the system.")
            return
        print("\n--- All Tasks ---")
        for task_id, task in self.tasks.items():
            print(f"ID: {task_id} | Name: {task.task_name} | Chargeable: {task.chargeable} | Rate: {task.rate_card}")
        print("-------------------")

# --- EMPLOYEE MANAGEMENT CLASSES ---
class Employee:

    def __init__(self, employee_id, employee_name, std_bill_rate, employee_address):
        self.employee_id = employee_id
        self.employee_name = self._validate_employee_name(employee_name)
        self.std_bill_rate = self._validate_std_bill_rate(std_bill_rate)
        self.employee_address = employee_address
    
    def _validate_employee_name(self, name):
        if not re.fullmatch(r"^[a-zA-Z0-9\s\-_]+$", name):
            raise InvalidInputError("Employee name can only contain alphabets, digits, '-', '_', and spaces.")
        return name
    
    def _validate_std_bill_rate(self, rate):
        if not isinstance(rate, (int, float)):
            try:
                rate = float(rate)
            except (ValueError, TypeError):
                raise InvalidInputError("Standard bill rate must be an integer or a float.")
        if rate < 0:
            raise InvalidInputError("Standard bill rate cannot be a negative value.")
        return rate
        
    def __repr__(self):
        return (f"Employee(id='{self.employee_id}', name='{self.employee_name}', "
                f"rate={self.std_bill_rate}, address={self.employee_address})")

class EmployeeManager:
    def __init__(self, storage_file="/Users/kaarunyalakshmanchinthalapudi/Downloads/employee.csv"):
        self.employees = {}
        self.storage_file = storage_file
        print(f"Loading employees from {self.storage_file}...")
    
    def _save_employees_to_file(self):
        print(f"Saving employees to {self.storage_file}...")
    
    def _generate_employee_id(self):
        while True:
            employee_id = f"EMP_{random.randint(100000, 999999)}"
            if employee_id not in self.employees:
                return employee_id

    def create_employee_from_csv(self, file_path="/Users/kaarunyalakshmanchinthalapudi/Downloads/employee.csv"):
        if not os.path.exists(file_path):
            print(f"Error: CSV file not found at {file_path}")
            return
            
        try:
            with open(file_path, 'r', encoding='latin-1') as csvfile:
                reader = csv.reader(csvfile)
                next(reader) 
                
                for i, row in enumerate(reader):
                    if len(row) < 12:
                        print(f"Skipping invalid row {i+1} from CSV: {row}")
                        continue
                    try:
                        employee_id = self._generate_employee_id()
                        
                        employee_name = row[0]
                        mail_id = row[1]
                        phone_number = row[2]
                        std_bill_rate = row[3]
                        house_no = row[4]
                        building_number = row[5]
                        road_number = row[6]
                        street_name = row[7]
                        land_mark = row[8]
                        city = row[9]
                        state = row[10]
                        zip_code = row[11]

                        address = Address(mail_id, phone_number, house_no=house_no, 
                                          building_number=building_number, road_number=road_number, 
                                          street_name=street_name, land_mark=land_mark, city=city, 
                                          state=state, zip_code=zip_code)
                        new_employee = Employee(employee_id, employee_name, std_bill_rate, address)
                        self.employees[new_employee.employee_id] = new_employee
                        print(f"Employee '{new_employee.employee_name}' created with ID {new_employee.employee_id} from CSV.")
                    except (InvalidInputError, IndexError) as e:
                        print(f"Error creating employee from CSV row {i+1}: {e}")
            self._save_employees_to_file()
        except Exception as e:
            print(f"An unexpected error occurred while reading the CSV file: {e}")

    def create_employee(self, employee_name, std_bill_rate, mail_id, phone_number,
                        house_no=None, building_number=None, road_number=None,
                        street_name=None, land_mark=None, city=None, state=None, zip_code=None):
        try:
            employee_id = self._generate_employee_id()
            address = Address(mail_id, phone_number, house_no, building_number, road_number,
                              street_name, land_mark, city, state, zip_code)
            new_employee = Employee(employee_id, employee_name, std_bill_rate, address)
            self.employees[new_employee.employee_id] = new_employee
            self._save_employees_to_file()
            return new_employee
        except InvalidInputError as e:
            print(f"Error creating employee: {e}")
            return None

    def update_employee(self, employee_id, **kwargs):
        if employee_id not in self.employees:
            raise EmployeeNotFoundError(f"Employee with ID {employee_id} not found.")
        employee = self.employees[employee_id]
        
        try:
            if 'std_bill_rate' in kwargs:
                employee.std_bill_rate = employee._validate_std_bill_rate(kwargs['std_bill_rate'])
            address_fields = ['mail_id', 'phone_number', 'house_no', 'building_number',
                              'road_number', 'street_name', 'land_mark', 'city',
                              'state', 'zip_code']
            for field in address_fields:
                if field in kwargs:
                    setattr(employee.employee_address, field, getattr(employee.employee_address, f"_validate_{field}", lambda x: x)(kwargs[field]))
            
            self._save_employees_to_file()
            print(f"Employee {employee_id} updated successfully.")
            return employee
        except InvalidInputError as e:
            print(f"Error updating employee {employee_id}: {e}")
            return None

    def delete_employee(self, employee_id):
        if employee_id not in self.employees:
            raise EmployeeNotFoundError(f"Employee with ID {employee_id} not found.")
        del self.employees[employee_id]
        self._save_employees_to_file()
        print(f"Employee with ID {employee_id} deleted successfully.")

    def search_employee(self, employee_id):
        return self.employees.get(employee_id)

    def list_all_employees(self):
        if not self.employees:
            print("No employees are available in the system.")
            return
        print("\n--- All Employees ---")
        for emp_id, emp in self.employees.items():
            print(f"ID: {emp_id} | Name: {emp.employee_name} | Rate: {emp.std_bill_rate} | Email: {emp.employee_address.mail_id}")
        print("---------------------")

# --- CLIENT MANAGEMENT CLASSES ---
class Client(Address):
    """Represents a client, which is a child class of Address."""
    def __init__(self, client_id, client_name, client_description, std_bill_rate, **address_kwargs):
        super().__init__(**address_kwargs)
        self.client_id = client_id
        self.client_name = self._validate_client_name(client_name)
        self.client_description = self._validate_client_description(client_description)
        self.std_bill_rate = self._validate_std_bill_rate(std_bill_rate)
    
    def _validate_client_name(self, name):
        if not re.fullmatch(r"^[a-zA-Z0-9\s\-_]+$", name):
            raise InvalidInputError("Client name can only contain alphabets, digits, '-', '_', and spaces.")
        return name

    def _validate_client_description(self, description):
        if not re.fullmatch(r"^[a-zA-Z0-9\s\-_]+$", description):
            raise InvalidInputError("Client description can only contain alphabets, digits, '-', '_', and spaces.")
        return description
    
    def _validate_std_bill_rate(self, rate):
        if not isinstance(rate, (int, float)):
            try:
                rate = float(rate)
            except (ValueError, TypeError):
                raise InvalidInputError("Standard bill rate must be an integer or a float.")
        if rate < 0:
            raise InvalidInputError("Standard bill rate cannot be a negative value.")
        return rate
        
    def __repr__(self):
        return (f"Client(id='{self.client_id}', name='{self.client_name}', "
                f"description='{self.client_description}', rate={self.std_bill_rate}, "
                f"address_mail='{self.mail_id}')")

class ClientManager:
    """Manages all client-related operations."""
    def __init__(self, storage_file="/Users/kaarunyalakshmanchinthalapudi/Downloads/clients.csv"):
        self.clients = {}
        self.storage_file = storage_file
        print(f"Loading clients from {self.storage_file}...")

    def _save_clients_to_file(self):
        print(f"Saving clients to {self.storage_file}...")

    def _generate_client_id(self):
        while True:
            client_id = f"CLT_{random.randint(100000, 999999)}"
            if client_id not in self.clients:
                return client_id

    def create_client_from_csv(self, file_path="/Users/kaarunyalakshmanchinthalapudi/Downloads/clients.csv"):
        if not os.path.exists(file_path):
            print(f"Error: CSV file not found at {file_path}")
            return
            
        try:
            with open(file_path, 'r', encoding='latin-1') as csvfile:
                reader = csv.reader(csvfile)
                next(reader) 
                
                for i, row in enumerate(reader):
                    if len(row) < 12:
                        print(f"Skipping invalid row {i+1} from CSV: {row}")
                        continue
                    try:
                        client_id = self._generate_client_id()
                        
                        client_name = row[0]
                        mail_id = row[1]
                        phone_number = row[2]
                        std_bill_rate = row[3]
                        house_number = row[4]
                        building_number = row[5]
                        road_number = row[6]
                        street_name = row[7]
                        landmark = row[8]
                        city = row[9]
                        state = row[10]
                        zip_code = row[11]
                        
                       
                        client_description = "No description available"

                        new_client = Client(client_id, client_name, client_description, std_bill_rate, 
                                            mail_id=mail_id, phone_number=phone_number, house_no=house_number, 
                                            building_number=building_number, road_number=road_number, 
                                            street_name=street_name, land_mark=landmark, city=city, 
                                            state=state, zip_code=zip_code)
                                            
                        self.clients[new_client.client_id] = new_client
                        print(f"Client '{new_client.client_name}' created with ID {new_client.client_id} from CSV.")
                    except (InvalidInputError, IndexError) as e:
                        print(f"Error creating client from CSV row {i+1}: {e}")
            self._save_clients_to_file()
        except Exception as e:
            print(f"An unexpected error occurred while reading the CSV file: {e}")
    
    def create_client(self, client_name, client_description, std_bill_rate, mail_id, phone_number,
                      house_no=None, building_number=None, road_number=None, street_name=None,
                      land_mark=None, city=None, state=None, zip_code=None):
        try:
            client_id = self._generate_client_id()
            new_client = Client(client_id, client_name, client_description, std_bill_rate, 
                                mail_id=mail_id, phone_number=phone_number, house_no=house_no, 
                                building_number=building_number, road_number=road_number, 
                                street_name=street_name, land_mark=land_mark, city=city, 
                                state=state, zip_code=zip_code)
            self.clients[new_client.client_id] = new_client
            self._save_clients_to_file()
            return new_client
        except InvalidInputError as e:
            print(f"Error creating client: {e}")
            return None

    def update_client(self, client_id, **kwargs):
        if client_id not in self.clients:
            raise ClientNotFoundError(f"Client with ID {client_id} not found.")
        client = self.clients[client_id]
        
        try:
            if 'client_description' in kwargs:
                client.client_description = client._validate_client_description(kwargs['client_description'])
            if 'std_bill_rate' in kwargs:
                client.std_bill_rate = client._validate_std_bill_rate(kwargs['std_bill_rate'])
            
            address_fields = ['mail_id', 'phone_number', 'house_no', 'building_number',
                              'road_number', 'street_name', 'land_mark', 'city',
                              'state', 'zip_code']
            for field in address_fields:
                if field in kwargs:
                    setattr(client, field, getattr(client, f"_validate_{field}", lambda x: x)(kwargs[field]))
            
            self._save_clients_to_file()
            print(f"Client {client_id} updated successfully.")
            return client
        except InvalidInputError as e:
            print(f"Error updating client {client_id}: {e}")
            return None

    def delete_client(self, client_id):
        if client_id not in self.clients:
            raise ClientNotFoundError(f"Client with ID {client_id} not found.")
        del self.clients[client_id]
        self._save_clients_to_file()
        print(f"Client with ID {client_id} deleted successfully.")

    def search_client(self, client_id):
        return self.clients.get(client_id)

    def list_all_clients(self):
        if not self.clients:
            print("No clients are available in the system.")
            return
        print("\n--- All Clients ---")
        for clt_id, clt in self.clients.items():
            print(f"ID: {clt_id} | Name: {clt.client_name} | Description: {clt.client_description} | Email: {clt.mail_id}")
        print("---------------------")
if __name__ == "__main__":
    task_manager = TaskManager()
    employee_manager = EmployeeManager()
    client_manager = ClientManager()
    tasks_csv_path = "/Users/kaarunyalakshmanchinthalapudi/Downloads/tasks.csv"
    employees_csv_path = "/Users/kaarunyalakshmanchinthalapudi/Downloads/employee.csv"
    clients_csv_path = "/Users/kaarunyalakshmanchinthalapudi/Downloads/clients.csv"

    print("\n--- TASK MANAGEMENT OPERATIONS ---")
    task_manager.create_task_from_csv(tasks_csv_path)
    task_manager.list_all_tasks()

    print("\n--- EMPLOYEE MANAGEMENT OPERATIONS ---")
    employee_manager.create_employee_from_csv(employees_csv_path)
    employee_manager.list_all_employees()

    print("\n--- CLIENT MANAGEMENT OPERATIONS ---")
    client_manager.create_client_from_csv(clients_csv_path)
    client_manager.list_all_clients()
    

Loading tasks from /Users/kaarunyalakshmanchinthalapudi/Downloads/tasks.csv...
Loading employees from /Users/kaarunyalakshmanchinthalapudi/Downloads/employee.csv...
Loading clients from /Users/kaarunyalakshmanchinthalapudi/Downloads/clients.csv...

--- TASK MANAGEMENT OPERATIONS ---
Error creating task from CSV row 1: Chargeable value must be 'True', 'False', 'Yes', or 'No'.
Task 'Documentation' created with ID TSK_263903 from CSV.
Task 'Investigation' created with ID TSK_734961 from CSV.
Task 'Business Settlement' created with ID TSK_155881 from CSV.
Task 'Tax' created with ID TSK_689480 from CSV.
Task 'Fallowups' created with ID TSK_474012 from CSV.
Task 'Diverse' created with ID TSK_811236 from CSV.
Task 'Vehical dealing' created with ID TSK_651843 from CSV.
Task 'Land Dealing' created with ID TSK_836552 from CSV.
Task 'Criminal Case dealing' created with ID TSK_307942 from CSV.
Task 'Finance settlement' created with ID TSK_837141 from CSV.
Task 'Client Communication' created with I